In [23]:

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


df = pd.read_csv('/content/drive/MyDrive/Dataset/Automobile.csv')


df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,name,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,70,usa
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,70,usa
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,70,usa
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,70,usa
4,ford torino,17.0,8,302.0,140.0,3449,10.5,70,usa


In [24]:
missing_data = df.isnull().sum()
print("Total Missing Value By Columns:")
print(missing_data)

Total Missing Value By Columns:
name            0
mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
dtype: int64


In [7]:
df['year'] = pd.to_datetime(df['year'])


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77878 entries, 0 to 77877
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   addref        77878 non-null  object        
 1   city          77876 non-null  object        
 2   assembly      24188 non-null  object        
 3   body          68974 non-null  object        
 4   make          77876 non-null  object        
 5   model         77876 non-null  object        
 6   year          73097 non-null  datetime64[ns]
 7   engine        77873 non-null  float64       
 8   transmission  77876 non-null  object        
 9   fuel          76970 non-null  object        
 10  color         76396 non-null  object        
 11  registered    77876 non-null  object        
 12  mileage       77876 non-null  float64       
 13  price         77293 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(10)
memory usage: 8.3+ MB


In [25]:
cols_to_use = ['horsepower','acceleration']
X = df[cols_to_use]  # Kolom fitur
y = df['weight']  # Kolom target


# Select target
y = df.weight

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)


my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [26]:
my_model = XGBRegressor(n_estimators=110)
my_model.fit(X_train, y_train)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 285.0073291015625


In [27]:

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(estimator=XGBRegressor(), param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)
predictions = best_model.predict(X_valid)
print("Updated Mean Absolute Error:", mean_absolute_error(predictions, y_valid))

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Updated Mean Absolute Error: 232.10088745117187


In [28]:

# Define the XGBoost model
model_grid = XGBRegressor()

# Define the parameter grid
param_grid = {
    'n_estimators': [150, 200, 250, 300, 1000],  # Adjust this range as needed
    'max_depth': [3, 4, 5],  # Add other hyperparameters to tune
    'learning_rate': [0.01, 0.1, 0.2]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model_grid, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error')

# Perform fitting using training data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)

# Use the best model for prediction
predictions = best_model.predict(X_valid)
print("Updated Mean Absolute Error:", mean_absolute_error(predictions, y_valid))

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000}
Updated Mean Absolute Error: 231.35029418945314
